# MLflow demo: list runs, plot metrics, register & load model

Purpose: show end-to-end MLflow usage (for portfolio / job applications).
- Connect to tracking store
- Inspect runs and metrics
- Plot F1 across runs
- Download `.skops` model artifact
- Register model in MLflow registry
- Load registered model


In [4]:
import mlflow
import pandas as pd
import skops.io as sio
import plotly.express as px
from pathlib import Path

# Point to your tracking store (adjust if remote)
TRACKING_URI = "file:./mlruns"
EXPERIMENT = "money-scam-mm"

mlflow.set_tracking_uri(TRACKING_URI)
mlflow.set_experiment(EXPERIMENT)

c:\Users\nuwai\Documents\Sophia_Skill_Development\Sophia_projects\Burmese Scam Detector\burmese_money_scam_classification\.venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
2026/01/05 19:50:46 INFO mlflow.tracking.fluent: Experiment with name 'money-scam-mm' does not exist. Creating a new experiment.


<Experiment: artifact_location=('file:///c:/Users/nuwai/Documents/Sophia_Skill_Development/Sophia_projects/Burmese '
 'Scam '
 'Detector/burmese_money_scam_classification/notebooks/mlruns/740748680789196737'), creation_time=1767619246017, experiment_id='740748680789196737', last_update_time=1767619246017, lifecycle_stage='active', name='money-scam-mm', tags={}>

## List runs and metrics
Filters to `best_run=true` and shows top runs; tweak `filter_string` as needed.

In [8]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT],
    filter_string="tags.best_run = 'true'",
    order_by=["metrics.weighted_f1_test DESC"],
)
print(runs)
#runs[['run_id','run_name','metrics.weighted_f1_test','metrics.weighted_f1_cv_mean']].head()

c:\Users\nuwai\Documents\Sophia_Skill_Development\Sophia_projects\Burmese Scam Detector\burmese_money_scam_classification\.venv\Lib\site-packages\mlflow\tracking\_tracking_service\utils.py:178: FutureWarning: The filesystem tracking backend (e.g., './mlruns') will be deprecated in February 2026. Consider transitioning to a database backend (e.g., 'sqlite:///mlflow.db') to take advantage of the latest MLflow features. See https://github.com/mlflow/mlflow/issues/18534 for more details and migration guidance. For migrating existing data, https://github.com/mlflow/mlflow-export-import can be used.
  return FileStore(store_uri, store_uri)
Traceback (most recent call last):
  File "c:\Users\nuwai\Documents\Sophia_Skill_Development\Sophia_projects\Burmese Scam Detector\burmese_money_scam_classification\.venv\Lib\site-packages\mlflow\store\tracking\file_store.py", line 379, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

Empty DataFrame
Columns: [run_id, experiment_id, status, artifact_uri, start_time, end_time]
Index: []


In [7]:
TRACKING_URI = r"file:///C:/Users/nuwai/Documents/Sophia_Skill_Development/Sophia_projects/Burmese Scam Detector/burmese_money_scam_classification/mlruns"
mlflow.set_tracking_uri(TRACKING_URI)


## Plot F1 across runs
Visualize both CV and test F1 for quick comparison.

In [ ]:
if runs.empty:
    print("No runs found. Adjust filters or run training first.")
else:
    plot_df = runs.copy()
    plot_df['run_display'] = plot_df['run_name'].fillna(plot_df['run_id'])
    fig = px.bar(
        plot_df,
        x='run_display',
        y=['metrics.weighted_f1_cv_mean','metrics.weighted_f1_test'],
        barmode='group',
        title='F1 scores across runs',
        labels={'value': 'F1', 'run_display': 'Run'}
    )
    fig.show()

## Download and load a `.skops` artifact
Pick a run (default: best_run first) and load the saved pipeline.

In [ ]:
if runs.empty:
    raise SystemExit("No runs to download from. Run training first.")

run_id = runs.iloc[0].run_id  # change to a specific run if desired
artifact_path = "model/best_model_tfidf.skops"  # adjust if your filename differs

local_path = mlflow.artifacts.download_artifacts(
    run_id=run_id,
    artifact_path=artifact_path,
)
loaded_model = sio.load(local_path)
loaded_model

## Register the model
Re-log the loaded model using MLflow's sklearn flavor so the registry can manage versions.

In [ ]:
REGISTERED_NAME = "scam-detector"  # change as needed

with mlflow.start_run(run_name="register_scam_detector"):
    mlflow.sklearn.log_model(
        loaded_model,
        artifact_path="model",
        registered_model_name=REGISTERED_NAME,
    )
    print("Registered model as:", REGISTERED_NAME)

## Load registered model
Load a specific version (or use `latest`).

In [ ]:
model_uri = f"models:/{REGISTERED_NAME}/1"  # use desired version or 'latest'
registered_model = mlflow.sklearn.load_model(model_uri)
registered_model